# Heart Attack - Regression

Source: https://www.kaggle.com/datasets/sukhmandeepsinghbrar/heart-attack-dataset?resource=download

- The heart attack datasets were collected at Zheen hospital in Erbil, Iraq, from January 2019 to May 2019. 

- The attributes of this dataset are: age, gender, heart rate, systolic blood pressure, diastolic blood pressure, blood sugar, ck-mb and troponin with negative or positive output. 

- According to the provided information, the medical dataset classifies either heart attack or none. 

- The gender column in the data is normalized: 

    - the male is set to 1 and the female to 0. 
    
    - The glucose column is set to 1 if it is > 120; otherwise, 0. 
    
    - As for the output, positive is set to 1 and negative to 0.

--- 

# Step 1: Import Data and Packages

In [ ]:
import numpy as np
import pandas as pd


import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv("Medicaldataset.csv")

data.head()

### Check columns

### Select Columns and Rows

### Train-Test Split

### Check the training set

### Check for missing data

--- 

# Step 2: Exploratory Data Analysis


### Correlation Matrix Heatmap

### Distributions

### Pairplot

---

# Step 3: Prepare the Data

### One-Hot Encode Categorical Features

### Check which features are continuous

### Feature Scaling on Continuous Features with Standardization

### Make sure it worked by seeing if Standard Deviations are 1

---

# Step 4: Linear Regression


### Full Model

### Backwards Elimination: Refit model with features of below 0.05 p-value

In [ ]:
---

# Step 5: Testing Interaction Terms

---

# Step 6: Ridge Regression
    
### GridSearchCV - Ridge

### Optimized Ridge Model

### GridSearchCV - Lasso

### Optimized Lasso Model

--- 
# Test Set Accuracy

---

# Final Conclusion
